In [1]:
import pyodbc
import pandas as pd

conn = pyodbc.connect(
    driver='FreeTDS',
    server='lga-db4.pulse.data',
    port=58321,
    uid='sqoopuser',
    pwd='sqoop@207*',
    tds_version='7.3'
)

query = """
SELECT * FROM Utility.dbo.ComparePage;
"""

df = pd.read_sql_query(query, conn)

In [2]:
df[:10]

,Dt,TableName,PaidImpressions,LastModifiedDt
0,2017-09-09,lga-hive-factlogevent,151447969,2017-10-09 17:21:00
1,2017-09-09,lga-hive-logevent,151447969,2017-10-09 17:21:00
2,2017-09-09,lga-sql-advDailyStats,151447969,2017-10-09 17:21:00
3,2017-09-09,lga-sql-KPIHourly,151447969,2017-10-09 17:21:00
4,2017-09-09,lga-sql-pubDailyStats,151447969,2017-10-09 17:21:00
5,2017-09-09,sjc-hive-factlogevent,88819475,2017-10-09 17:21:00
6,2017-09-09,sjc-hive-logevent,151447969,2017-10-09 17:21:00
7,2017-09-09,sjc-sql-advDailyStats,151447969,2017-10-09 17:21:00
8,2017-09-09,sjc-sql-KPIHourly,151447969,2017-10-09 17:21:00
9,2017-09-09,sjc-sql-pubDailyStats,151447969,2017-10-09 17:21:00


In [3]:
list(df.TableName.unique())     # len is 12

[u'lga-hive-factlogevent',
 u'lga-hive-logevent',
 u'lga-sql-advDailyStats',
 u'lga-sql-KPIHourly',
 u'lga-sql-pubDailyStats',
 u'sjc-hive-factlogevent',
 u'sjc-hive-logevent',
 u'sjc-sql-advDailyStats',
 u'sjc-sql-KPIHourly',
 u'sjc-sql-pubDailyStats',
 u'vertica-kpihourly',
 u'vertica-rpt']

In [4]:
pivoted = pd.pivot_table(df, values='PaidImpressions', index=['Dt'], columns=['TableName'])

In [8]:
df_descending_dates = pivoted.sort_index(ascending=False, kind='mergesort')

In [9]:
smalldf = df_descending_dates.head()
smalldf.index

Index([2017-10-09, 2017-10-08, 2017-10-07, 2017-10-06, 2017-10-05], dtype='object', name=u'Dt')

In [13]:
smalldf.shape

(5, 12)

In [11]:
smalldf.to_csv(sep='\t').strip('\n')

'Dt\tlga-hive-factlogevent\tlga-hive-logevent\tlga-sql-KPIHourly\tlga-sql-advDailyStats\tlga-sql-pubDailyStats\tsjc-hive-factlogevent\tsjc-hive-logevent\tsjc-sql-KPIHourly\tsjc-sql-advDailyStats\tsjc-sql-pubDailyStats\tvertica-kpihourly\tvertica-rpt\n2017-10-09\t88432281.0\t88432281.0\t116954130.0\t72263486.0\t72263486.0\t64051179.0\t80382732.0\t88432281.0\t72263486.0\t72263486.0\t116954130.0\t\n2017-10-08\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\n2017-10-07\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\n2017-10-06\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\n2017-10-05\t158079882.0\t158079882.0\t158079882.0\t158079882.0\t158079882.0\t158079882.0\t1580798

In [12]:
smalldf.to_csv(sep='\t')

'Dt\tlga-hive-factlogevent\tlga-hive-logevent\tlga-sql-KPIHourly\tlga-sql-advDailyStats\tlga-sql-pubDailyStats\tsjc-hive-factlogevent\tsjc-hive-logevent\tsjc-sql-KPIHourly\tsjc-sql-advDailyStats\tsjc-sql-pubDailyStats\tvertica-kpihourly\tvertica-rpt\n2017-10-09\t88432281.0\t88432281.0\t116954130.0\t72263486.0\t72263486.0\t64051179.0\t80382732.0\t88432281.0\t72263486.0\t72263486.0\t116954130.0\t\n2017-10-08\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\t152502300.0\n2017-10-07\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\t146430539.0\n2017-10-06\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\t158814386.0\n2017-10-05\t158079882.0\t158079882.0\t158079882.0\t158079882.0\t158079882.0\t158079882.0\t1580798